In [1]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:
from sklearn.preprocessing import MinMaxScaler


In [3]:
data = pd.read_csv('daily-website-visitors.csv')


In [4]:
data

,Row,Day,Day.Of.Week,Date,Page.Loads,Unique.Visits,First.Time.Visits,Returning.Visits
0,1,Sunday,1,9/14/2014,2146,1582,1430,152
1,2,Monday,2,9/15/2014,3621,2528,2297,231
2,3,Tuesday,3,9/16/2014,3698,2630,2352,278
3,4,Wednesday,4,9/17/2014,3667,2614,2327,287
4,5,Thursday,5,9/18/2014,3316,2366,2130,236
...,...,...,...,...,...,...,...,...
2162,2163,Saturday,7,8/15/2020,2221,1696,1373,323
2163,2164,Sunday,1,8/16/2020,2724,2037,1686,351
2164,2165,Monday,2,8/17/2020,3456,2638,2181,457
2165,2166,Tuesday,3,8/18/2020,3581,2683,2184,499


In [5]:
data = data.filter(['Page.Loads'])


In [6]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,Page.Loads
1167,2658
1168,2391
1169,3202
1170,4349
1171,4886
...,...
2162,2221
2163,2724
2164,3456
2165,3581


In [7]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data['Page.Loads'].values.reshape(-1,1))

In [8]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [9]:
#Chia dữ liệu thành train test do chỉ lấy tập train để tìm ra tham số tốt nhất
data = train_data

train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [10]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 90
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [11]:
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

# Khởi tạo một số tham số
input_shape = (90, 1)
batch_size = 16
epochs = 10000
num_filters = [32] # các số lượng filter khác nhau [32, 64, 128, 256]
kernel_sizes = [2] # các kích thước kernel khác nhau [2,3,4,6,8,16,32]
scores = [] # list để lưu kết quả đánh giá mô hình


# Load dữ liệu
# Thực hiện tìm kiếm lưới
for num_filter in num_filters:
    for kernel_size in kernel_sizes:
        # Khởi tạo mô hình với số lớp convolution và kích thước kernel tương ứng
        model = Sequential()
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu'))
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu'))
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu'))
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu'))

        #MaxPooling là một phép lọc giá trị lớn nhất từ các kết quả của lớp convolution trước 
        model.add(MaxPooling1D(pool_size=2))
        #Đưa về dạng 1 chiều 
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        # Lưu thời điểm bắt đầu huấn luyện
        start_time = time.time()
        # Huấn luyện mô hình
        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_test, y_test))
        # Đánh giá kết quả của mô hình và lưu vào scores
        y_pred = model.predict(X_test)
        y_pred = np.reshape(y_pred, (-1, 1))
        # Lưu thời điểm kết thúc huấn luyện
        end_time = time.time()
        # Tính thời gian huấn luyện
        training_time = end_time - start_time
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mape = (mean_absolute_percentage_error(y_test, y_pred))*100
        
        scores.append((num_filter, kernel_size, mse, rmse, mape, training_time))
        
# Hiển thị kết quả tìm kiếm lưới
for score in scores:
    print("num_filters=%d, kernel_size=%d, mse=%.5f, rmse=%.5f, mape=%.5f, training_time=%.2f seconds" % (score[0], score[1], score[2], score[3], score[4],score[5]))



Epoch 1/10000
25/25 [==============================] - 1s 13ms/step - loss: 0.0677 - val_loss: 0.0128
Epoch 2/10000
25/25 [==============================] - 0s 5ms/step - loss: 0.0253 - val_loss: 0.0072
Epoch 3/10000
25/25 [==============================] - 0s 6ms/step - loss: 0.0183 - val_loss: 0.0042
Epoch 4/10000
25/25 [==============================] - 0s 6ms/step - loss: 0.0143 - val_loss: 0.0073
Epoch 5/10000
25/25 [==============================] - 0s 6ms/step - loss: 0.0139 - val_loss: 0.0080
Epoch 6/10000
25/25 [==============================] - 0s 6ms/step - loss: 0.0102 - val_loss: 0.0054
Epoch 7/10000
25/25 [==============================] - 0s 6ms/step - loss: 0.0085 - val_loss: 0.0068
Epoch 8/10000
25/25 [==============================] - 0s 5ms/step - loss: 0.0080 - val_loss: 0.0030
Epoch 9/10000
25/25 [==============================] - 0s 5ms/step - loss: 0.0069 - val_loss: 0.0085
Epoch 10/10000
25/25 [==============================] - 0s 5ms/step - loss: 0.0069 - val_l

KeyboardInterrupt: 